In [1]:
import os
os.chdir('/home/victorhuang/projects/gtx/')

import numpy as np
import torch

from kan import *
from code_kan.kan_preprocess import load_data


In [2]:
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("Number of GPUs:", torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print(f"\nGPU {i}:")
        print("  Name:", torch.cuda.get_device_name(i))
        print("  Memory Allocated:", round(torch.cuda.memory_allocated(i) / 1024**2, 2), "MB")
        print("  Memory Cached:", round(torch.cuda.memory_reserved(i) / 1024**2, 2), "MB")
        print("  Capability:", torch.cuda.get_device_capability(i))
        print("  Current Device:", torch.cuda.current_device())
    device = torch.device('cuda')
else:
    print("No GPU available.")
    device = torch.device('cpu')

PyTorch version: 2.2.2+cu121
CUDA available: True
Number of GPUs: 1

GPU 0:
  Name: NVIDIA GeForce RTX 4070 Laptop GPU
  Memory Allocated: 0.0 MB
  Memory Cached: 0.0 MB
  Capability: (8, 9)
  Current Device: 0


# load data

In [3]:
dt_data_path = 'data/20241118_data_splited.mat'
data = load_data(dt_data_path, data_type='dt')

train_data = data['train']
train_fluorescence = train_data['fluorescence'].transpose(0, 2, 3, 1)
train_op = train_data['op'].transpose(0, 2, 3, 1)
train_depth = train_data['depth']
train_concentration_fluor = train_data['concentration_fluor']
train_reflectance = train_data['reflectance']

val_data = data['val']
val_fluorescence = val_data['fluorescence'].transpose(0, 2, 3, 1)
val_op = val_data['op'].transpose(0, 2, 3, 1)
val_depth = val_data['depth']
val_concentration_fluor = val_data['concentration_fluor']
val_reflectance = val_data['reflectance']

test_data = data['test']
test_fluorescence = test_data['fluorescence'].transpose(0, 2, 3, 1)
test_op = test_data['op'].transpose(0, 2, 3, 1)
test_depth = test_data['depth']
test_concentration_fluor = test_data['concentration_fluor']
test_reflectance = test_data['reflectance']


# KAN

In [4]:
dataset = {}
dataset['train_input'] = torch.cat((torch.tensor(test_fluorescence), torch.tensor(test_op)), dim=3).reshape(-1, 8).to(device)
dataset['train_label'] = torch.tensor(test_depth).reshape(-1, 1).to(device)
dataset['test_input'] = torch.cat((torch.tensor(val_fluorescence), torch.tensor(val_op)), dim=3).reshape(-1, 8).to(device)
dataset['test_label'] = torch.tensor(val_depth).reshape(-1, 1).to(device)

print(dataset['train_input'].shape)
print(dataset['train_label'].shape)
print(dataset['test_input'].shape)
print(dataset['test_label'].shape)

torch.Size([10201000, 8])
torch.Size([10201000, 1])
torch.Size([10201000, 8])
torch.Size([10201000, 1])


In [5]:
model = KAN(width=[8, 8, 1], grid=8, k=2, seed=42, device=device)

checkpoint directory created: ./model
saving model version 0.0


In [6]:
model.fit(dataset, opt="LBFGS", steps=10)

description:   0%|                                                           | 0/10 [01:29<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.35 GiB. GPU 0 has a total capacity of 8.00 GiB of which 0 bytes is free. Process 11730 has 17179869184.00 GiB memory in use. Of the allocated memory 17.11 GiB is allocated by PyTorch, and 2.82 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)